In [17]:
from kubernetes import client, config
import subprocess
import json 
from datetime import datetime
import threading

import csv   

fields=['timestamp', 'pod','cpu','memory', 'window']

# ['usage'] 

def getMetrics(name):
    result = subprocess.run(['kubectl', 'get', '--raw', '/apis/metrics.k8s.io/v1beta1/namespaces/sim/pods/' + name], stdout=subprocess.PIPE)
    metric = json.loads(result.stdout)
    return metric['containers'][0]

class StoppableThread(threading.Thread):
    """Thread class with a stop() method. The thread itself has to check
    regularly for the stopped() condition."""

    def __init__(self):
        super(StoppableThread, self).__init__()
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()

class CollectionThread(StoppableThread):

    def __init__(self, performanceFile):
        super().__init__()
        self.performanceFile = performanceFile


    def run(self):
        # Configs can be set in Configuration class directly or using helper utility
        config.load_kube_config()

        v1 = client.CoreV1Api()
        ret = v1.list_pod_for_all_namespaces(watch=False)

        with open(self.performanceFile, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
            while not self.stopped():
                timestamp = datetime.now()
                for i in ret.items:
                    if (i.metadata.namespace == "sim"):
                        m = getMetrics(i.metadata.name)
                        writer.writerow([timestamp, i.metadata.name, int(m['usage']['cpu'][:-1]), int(m['usage']['memory'][:-2]), 0])
            print("stopped")

In [18]:
import threading
import time
import os
import shutil

output = "distributionOfWorkload"
version = "v1"

outputDir = os.path.join(output, version)

if not os.path.exists(outputDir):
    os.mkdir(outputDir)

performanceFile = os.path.join(outputDir, 'podPerformance.csv')
resultFile = os.path.join(outputDir, 'result.jtl')

if os.path.exists(performanceFile):
    os.remove(performanceFile)

if os.path.exists(resultFile):
    os.remove(resultFile) 


collection_thread = CollectionThread(performanceFile)
collection_thread.start()


subprocess.run(['jmeter', '-n', '-t', 'jmeterRunGetItem.jmx', '-f', '-l', resultFile])

collection_thread.stop()

Creating summariser <summary>
Created the tree successfully using jmeterRunGetItem.jmx
Starting standalone test @ Thu Jul 21 15:39:09 CEST 2022 (1658410749276)
Waiting for possible Shutdown/StopTestNow/HeapDump/ThreadDump message on port 4445
summary +    126 in 00:00:21 =    6.1/s Avg:  2232 Min:   288 Max:  6196 Err:     0 (0.00%) Active: 20 Started: 20 Finished: 0
summary +    200 in 00:00:30 =    6.6/s Avg:  2850 Min:   579 Max:  6077 Err:     0 (0.00%) Active: 20 Started: 20 Finished: 0
summary =    326 in 00:00:51 =    6.4/s Avg:  2611 Min:   288 Max:  6196 Err:     0 (0.00%)
summary +    219 in 00:00:30 =    7.3/s Avg:  2823 Min:   576 Max:  6614 Err:     0 (0.00%) Active: 11 Started: 20 Finished: 9
summary =    545 in 00:01:21 =    6.7/s Avg:  2696 Min:   288 Max:  6614 Err:     0 (0.00%)
summary +     61 in 00:00:09 =    6.7/s Avg:   964 Min:   196 Max:  3086 Err:     0 (0.00%) Active: 0 Started: 20 Finished: 20
summary =    606 in 00:01:30 =    6.7/s Avg:  2522 Min:   196 Max

0 2 1 3 4 0 = std

0 10 0 0 0 = std

0 5 5 0 0 0 = std

